In [1]:
import pandas as pd

## Load Dataset

In [2]:
df = pd.read_csv('train_quora.csv',encoding='ISO-8859-1')

## Randomly Sample 150 negative samples and 50 positive samples

In [5]:
from numpy.random import choice
draw1 = choice(df[df.target==0].index, 150)
draw2 = choice(df[df.target==1].index, 50)

In [6]:
draw = list(draw1) + list(draw2)
draw=list(set(draw))

In [7]:
df_sample_200=df.loc[draw]

In [8]:
sum(df_sample_200.target==1)

50

In [9]:
sum(df_sample_200.target==0)

150

## Create a new DataFrame to pass into SageMaker

In [10]:
df_final=pd.DataFrame(df_sample_200)

In [11]:
df_final.head()

,qid,question_text,target
234496,2dde2b5407db67968b99,What did the rural pupils are not much the hap...,0.0
147457,1cd6ebc29e2b6d4456d4,Should we mandate minimum qualifications for p...,0.0
251916,314ce3dca824bfbd910b,Is South Africa doomed to become the next Zimb...,1.0
413203,50f6d89a243e5bbc0085,Why Indians love Kashmir but not Kashmiris?,1.0
32279,065182a62cead0dd2d65,What is the importency of hamalya mountain?,0.0


In [12]:
#df_final.to_excel('quora_200_sample.xlsx','Sheet1')

## Format Data for SageMaker

In [13]:
articles_prepare_for_sagemaker = []
articles_out_of_range=[]
for i in df_final.index:
    temp='is this question insincere? '+ ' [question: ' + df.question_text.values[i] +' ] ' + ' [id: '+ str(df.qid[i]) +' ]'   
    
    temp=temp.replace('\r\n', ' ').replace('\n\n',' ').replace('\n',' ').replace('\r',' ').replace('\r\r',' ')
    #aws sagemarker will not work on article length greater than 100,000
    if len(temp) <= 100000:
        articles_prepare_for_sagemaker.append(temp)
    else:
        articles_out_of_range.append(i)
        

In [15]:
articles_out_of_range

[]

In [16]:
articles_prepare_for_sagemaker[0]

'is this question insincere?  [question: What did the rural pupils are not much the happiness? ]  [id: 2dde2b5407db67968b99 ]'

In [17]:
articles_prepare_for_sagemaker=pd.DataFrame(articles_prepare_for_sagemaker)

In [18]:
articles_prepare_for_sagemaker.shape

(200, 1)

In [19]:
articles_prepare_for_sagemaker.loc[4][0]

'is this question insincere?  [question: What is the importency of hamalya mountain? ]  [id: 065182a62cead0dd2d65 ]'

In [20]:
articles_prepare_for_sagemaker.shape

(200, 1)

## Output an CSV file

In [34]:
articles_prepare_for_sagemaker.to_csv('quora-sample-200.csv',index=False,header=False)